#  Chapter 3

In [1]:
import numpy as np
from numpy.linalg import *
from math import pi, sqrt
import scipy as sp
from sympy import simplify

import matplotlib; matplotlib.use("TkAgg") #THIS IS KEY TO ANIMATIONS WORKING
import matplotlib.pyplot as plt

from spatialmath.base import *
from spatialmath.base import sym
from spatialmath import SE3, SO2, SO3, UnitQuaternion
from roboticstoolbox import tpoly, lspb, mtraj, qplot, ctraj

np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%config ZMQInteractiveShell.ast_node_interactivity = 'last_expr_or_assign'


In [6]:
J = np.array([[2, -1, 0], [-1, 4, 0], [0, 0, 3]])
dt = 0.05
def attitude():
   attitude = UnitQuaternion()
   w = 0.2 * np.r_[1, 2, 2].T
   for t in np.arange(0, 10, dt):
      wd =  -np.linalg.inv(J) @ (np.cross(w, J @ w))
      w += wd * dt
      attitude.increment(w * dt)
      yield attitude.R
    
plotvol3(2)
tranimate(attitude())


calling run
returning from run


# Sec 3.1 Time-varying pose
## Sec. 3.1.2 transforming spatial velocity

In [7]:
aTb = SE3(-2, 0, 0) * SE3.Rz(-pi/2) * SE3.Rx(pi/2)

SE3:   0           0          -1          -2            
      -1           0           0           0            
       0           1           0           0            
       0           0           0           1            
    

In [8]:
wT = SE3()
aTb = SE3(-2, 0, 0) * SE3.Rz(-pi/2) * SE3.Rx(pi/2);

plotvol3(5)
wT.plot(frame='A', color='b')
aTb.plot(frame='B', color='r')

In [9]:
# body to world frame
bV = [1, 2, 3, 4, 5, 6]

[1, 2, 3, 4, 5, 6]

In [10]:
aV = aTb.jacob() @ bV

array([      -3,       -1,        2,       -6,       -4,        5])

In [11]:
# same body
aV = aTb.Ad() @ [1, 2, 3, 0, 0, 0]

array([      -3,       -1,        2,        0,        0,        0])

In [12]:
aV = aTb.Ad() @ [0, 0, 0, 1, 0, 0]

array([       0,        0,        2,        0,       -1,        0])

In [13]:
aV = aTb.Ad() @ [1, 2, 3, 1, 0, 0]

array([      -3,       -1,        4,        0,       -1,        0])

# 3.1.3 incremental rotation

In [14]:
rotx(0.001)

array([[       1,        0,        0],
       [       0,        1,   -0.001],
       [       0,    0.001,        1]])

In [15]:
R1 = np.eye(3,3);
R1cheap = np.eye(3,3);
w = np.r_[1, 0, 0]
dt = 0.01
for i in range(100):
   R1 = R1 @ trexp(skew(w*dt))
   R1cheap = R1cheap + R1cheap @ skew(w*dt)

In [16]:
np.linalg.det(R1) - 1

-2.886579864025407e-15

In [17]:
np.linalg.det(R1cheap) - 1

0.010049662092876055

In [18]:
tr2angvec(trnorm(R1))

(1.0, array([       1,        0,        0]))

In [19]:
tr2angvec(trnorm(R1cheap))

(0.9999666686665241, array([       1,        0,        0]))

# 3.2.1 dynamics of moving bodies

In [20]:
J = np.array([[2, -1, 0], [-1, 4, 0], [0, 0, 3]])

array([[ 2, -1,  0],
       [-1,  4,  0],
       [ 0,  0,  3]])

In [21]:
dt = 0.05

0.05

In [22]:
def attitude():
   attitude = UnitQuaternion()
   w = 0.2 * np.r_[1, 2, 2].T
   for t in np.arange(0, 10, dt):
      wd =  -np.linalg.inv(J) @ (np.cross(w, J @ w))
      w += wd * dt
      attitude.increment(w * dt)
      yield r2t(attitude.R)

In [23]:
plt.figure()
plotvol3(2)
tranimate(attitude())

calling run
returning from run


# 3.2.2 transforming forces/torques
wrench example

In [24]:
aW = aTb.inv().Ad().T @ [1, 2, 3, 0, 0, 0]

array([      -3,       -1,        2,        0,        4,        2])

# 3.3 smooth 1D trajectories

In [25]:
traj = tpoly(0, 1, np.linspace(0, 1, 50))
traj.plot()

In [26]:
traj2 = tpoly(0, 1, np.linspace(0, 1, 50), 10, 0)
traj2.plot()

In [27]:
np.mean(traj.yd) / np.max(traj.yd)

0.523102222222222

In [28]:
traj = lspb(0, 1, np.linspace(0, 1, 50))
traj.plot()

In [29]:
np.mean(traj.yd) / np.max(traj.yd)

0.653061224489796

In [31]:
traj = lspb(0, 1, np.linspace(0, 1, 50))
traj.plot()

In [33]:
traj = lspb(0, 1, np.linspace(0, 1, 50), 1.2)
traj.plot()

In [34]:
np.max(traj.yd)

1.2

# 3.3.2 multi-dimensional case

In [35]:
q = mtraj(lspb, [0, 2], [1, -1], 50)
qplot(q.y)
# HACK q = [T1.t' T1.torpy]

In [ ]:
## 3.3.3 multi-segment trajectoreis
via = SO2(30, unit='deg') * np.array([[-1, 1, 1, -1, -1], [1, 1, -1, -1, 1]])

In [ ]:
traj0 = mstraj(via.T, dt=0.2, tacc=0, qdmax=[2, 1])

In [ ]:
plt.plot(traj0.q[:,0], traj0.q[:,1])

In [ ]:
traj2 = mstraj(via.T, dt=0.2, tacc=2, qdmax=[2, 1])

HACK q0 = mstraj(via(:,[2 3 4 1])', [2,1], [], via(:,1)', 0.2, 0)
HACK plot(q0[:,0], q0[:,1])

HACK q2 = mstraj(via(:,[2 3 4 1])', [2,1], [], via(:,1)', 0.2, 2)

In [ ]:
[len(traj0), len(traj2)]

In [ ]:
## 3.3.4 interpotation of orientation
R0 = SO3.Rz(-1) * SO3.Ry(-1)
R1 = SO3.Rz(1) * SO3.Ry(1)

In [ ]:
rpy0 = R0.rpy()
rpy1 = R1.rpy()
rpy = mtraj(tpoly, rpy0, rpy1, 50)
pose = SO3.RPY(rpy.y)
len(pose)
plotvol3(2); pose.animate()

In [ ]:
q0 = UnitQuaternion(R0)
q1 = UnitQuaternion(R1)

In [ ]:
# HACK
q = q0.interp(q1, 50)
len(q)
plotvol3(2); q.animate()

In [ ]:
## 3.3.5 direction of rotation
q0 = UnitQuaternion.Rz(-2)
q1 = UnitQuaternion.Rz(2)
q = q0.interp(q1, 50)
q.animate()

In [ ]:
q = q0.interp(q1, 50, shortest=True)
q.animate()

In [ ]:
## 3.3.6 Cartesian motion
T0 = SE3([0.4, 0.2, 0]) * SE3.RPY([0, 0, 3])
T1 = SE3([-0.4, -0.2, 0.3]) * SE3.RPY([-pi/4, pi/4, -pi/2])

In [ ]:
T0.interp(T1, 0.5)

In [ ]:
Ts = T0.interp(T1, 50)

In [ ]:
len(Ts)

In [ ]:
Ts[0]

In [ ]:
plotvol3(2)
Ts.animate()
P = Ts.t
P.shape
qplot(P)

In [ ]:
rpy = Ts.rpy()
qplot(rpy)

In [ ]:
Ts = T0.interp(T1,lspb(0, 1, 50).y)
Ts = ctraj(T0, T1, 50)

# 3.4.1.2 estimating orientation

In [ ]:
from imu_data import IMU
true, imu = IMU()

In [ ]:
attitude = UnitQuaternion()
for w in true.omega[:-1,:]:
   attitude.append(attitude[-1] * UnitQuaternion.EulerVec(w*true.dt))

In [ ]:
print(attitude[99])
attitude.animate(true=imu.t)

In [ ]:
plt.clf()
plt.plot(true.t, attitude.rpy())

In [ ]:
## 3.4.4 sensor fusion
from imu_data import IMU
true, imu = IMU()
t = imu.t
attitude_naive = UnitQuaternion()
for w in imu.gyro[:-1]:
   attitude_naive.append(attitude_naive[-1] * UnitQuaternion.EulerVec(w * imu.dt))

In [ ]:
plt.clf()
plt.plot(attitude_naive.rpy())
plt.title('naive')
plt.figure()
plt.plot(true.attitude.rpy())
plt.title('true')
plt.figure()
plt.plot(t, attitude_naive.angdist(true.attitude, metric=1), 'r' )

In [ ]:
kI = 0.2
kP =1

In [ ]:
b = np.zeros(imu.gyro.shape)  # initial bias
attitude_ECF = UnitQuaternion()

In [ ]:
for k, (wm, am, mm) in enumerate(zip(imu.gyro[:-1], imu.accel[:-1], imu.magno[:-1])):
   invq = attitude_ecf[k].inv()
   sigmaR = np.cross(am, invq * true.g) + np.cross(mm, invq * true.B)
   wp = wm - b[k,:] + kP * sigmaR
   attitude_ECF.append(attitude_ECF[k] * UnitQuaternion.EulerVec(wp * imu.dt))
   b[k+1,:] = b[k,:] - kI * sigmaR * imu.dt

In [ ]:
plt.plot(t, attitude_ECF.angdist(true.attitude, metric=1), 'b')
plt.xlim(0, 20)
plt.ylim(0, 0.8)
plt.legend('Naive integration', 'ECF')

In [ ]:
plt.figure()
plt.plot(t, b)
plt.show(block=True)